# Working with Excel files
Excel is the "Swiss army knife" of business analytics. For those of you who have taken my **MIS 4460/5460 - Business Analytics** course, you know that Excel is extraordinarily powerful and has numerous features and capabilities that the vast majority of users (even those that think they are pretty good with Excel) don't even know exist. 

## Installing Python packages
Okay, first we need to talk about the whole issue of package management. This is a little confusing in general, and even more so when using the Anaconda Python distro. 

### Pretend we aren't using Anaconda for a moment
While Python comes with a ton of libraries built in to the standard library, there are thousands of external libraries or *packages* available for you to use. These packages are collected in an online repository called PyPI (the Python Package Index) at https://pypi.python.org/pypi. Let's go there and get a sense of the layout of the site. 

Let's search for packages related to Excel. There are thousands :(. In this tutorial we are going to use one of these packages:

* [openpyxl](https://openpyxl.readthedocs.io/en/stable/) - Read and write xlsx files

To install packages that live in PyPI, we use another Python package called `pip` (https://pip.pypa.io/en/stable/ and https://en.wikipedia.org/wiki/Pip_(package_manager)). 

To install a Python package from PyPI using `pip` such as `openpyxl`, all we have to do is open a shell window and type: **DON'T DO THIS**

    pip install openpyxl
    
A bunch of stuff will scroll down the screen as `pip` figures out all dependencies for the package you want to install and eventually will prompt you for confirmation on continuing. Once a package is successfully installed, you can use it. 

To uninstall:

    pip uninstall openpyxl
    
While installing and uninstalling packages with `pip` is pretty easy, finding the right package is a little trickier. There are often multiple packages for doing the same or similar things. And of course, the quality of packages can vary tremendously. It's a giant sea of open source software submitted by a sea of people. I always do a bunch of Googling and reading to figure out which packages seem to get recommended by others and are under active development or at least have long term stability. Let's put "python read Excel files" into Google and see what we find. I can tell you one thing you'll find - there are a lot of really dedicated and helpful Python developers and power users who write all kinds of blogs, tutorials, and forum posts to share their knowledge. One of the great benefits of using open source software is the communities and ecosystems that grow around many software packages (like Python).

One particularly nice tutorial on this topic is https://www.datacamp.com/community/tutorials/python-excel-tutorial.

### But we are using Anaconda Python

While we can use `pip` with Anaconda, it comes with its own package management program called `conda` (http://conda.pydata.org/docs/intro.html). If you want an overview on conda virtual environments as well as the details, see:

- [Getting started with Python environments (using Conda)](https://towardsdatascience.com/getting-started-with-python-environments-using-conda-32e9f2779307)
- [Overview from Anaconda User Guide](https://conda.io/projects/conda/en/latest/user-guide/concepts/environments.html)
- [Managing Environments page with all the details](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html)



Conda is used for much more than simply installing and uninstalling packages. As you become more experienced with Python, you'll start to learn about things like using *virtual environments* and even creating and distributing your own packages. Here's the first two paragraphs from the Conda documentation:

> Conda is a package manager application that quickly installs, runs, and updates packages and their dependencies. The conda command is the primary interface for managing installations of various packages. It can query and search the package index and current installation, create new environments, and install and update packages into existing conda environments. See our Using conda section for more information.

>Conda is also an environment manager application. A conda environment is a directory that contains a specific collection of conda packages that you have installed. For example, you may have one environment with NumPy 1.7 and its dependencies, and another environment with NumPy 1.6 for legacy testing. If you change one environment, your other environments are not affected. You can easily activate or deactivate (switch between) these environments. You can also share your environment with someone by giving them a copy of your environment.yaml file.

Turns out we can even mix in `pip` based installs with our `conda` based packages. We'll likely run into this at some point when we need a package that's not available through the Anaconda Python distro or the Anaconda.org site (like PyPI but for Conda). However, mixing `pip` and `conda` installs can lead to a bit of a mess. In general, try to install packages using `conda` if you are using the Anaconda Python distribution. To see all of the packages installed in the current conda virtual environment (or the base environment), just do the following:

In [ ]:
!conda list


You'll notice that both `openpyxl` is already installed. If it isn't, perhaps I created a conda virtual env (see below) and didn't include it. To install it (if needed), we use the shell (an Anaconda Prompt in Windows) and type:

    conda activate datasci
    conda install openpyxl
    

# Example 1: Processing data across multiple worksheets
I mentioned in class that I was involved in some research projects in which stream temperature loggers are used to gather temperature data in a bunch of streams throughout the world.

- http://advances.sciencemag.org/content/5/1/eaav0486
- http://www.geminidataloggers.com/data-loggers/tinytag-plus-2/tgp-4017
- https://www.maximintegrated.com/en/products/ibutton/ibuttons/thermochron.cfm

Scientists download the data from the logger into either CSV or Excel files. The data file format can vary. It's a very challenging data cleaning and standardizing problem. For now, we'll consider one specific file type that gets submitted - a single Excel file with each logger's output living in its own sheet within the data file.

The file `data/sites_1_6.xlsx` contains six sheets named Site1, Site2, ..., Site6. Each sheet contains two columns:

    datetime	            temp_c
    09/15/15 12:00:00 PM	10.22
    09/15/15 01:00:00 PM	10.32
    09/15/15 02:00:00 PM	10.44
    09/15/15 03:00:00 PM	10.49
    09/15/15 04:00:00 PM	10.57
    09/15/15 05:00:00 PM	10.61
    ...

We aren't sure how many rows each file has.

Let's use Python to get the data from each sheet, add a new column with the name of the site, and consolidate all six sheets into a single set of records that we can write out as a CSV file.

We will use the `openpyxl` library. It appears to be the recommended library for XLSX files (see http://www.python-excel.org/).

In [1]:
import openpyxl

Start by opening the workbook.

In [2]:
wb_sites = openpyxl.load_workbook('data/sites_1_6.xlsx')
print(type(wb_sites))

<class 'openpyxl.workbook.workbook.Workbook'>


Our basic strategy will be to:

- iterate through the worksheets
- collect the rows of each worksheet into lists (one list per worksheet)
- store these lists in a main container list
- write out the container list of lists to a CSV file

In [3]:
 # Accumulate the rows (lists) into another list. We'll end up with a list of lists.
data = [] # Container list

# Iterate over the worksheets in the workbook
for ws in wb_sites:
    # Grab the worksheet name to use as a column in each row
    site = ws.title

    # Iterate over rows for the current sheet
    for row in ws.iter_rows():
        # Create a list containing the cell value in current row
        ws_data = [cell.value for cell in row]
        # Append the worksheet name to the row (so we know where the row came from)
        ws_data.append(site)
        # Append this list (one row of data) to the container list
        data.append(ws_data)

Take a look at the first few rows and the last few rows.

In [4]:
data[:10]

[['datetime', 'temp_c', 'Site1'],
 ['09/15/15 12:00:00 PM', 10.222, 'Site1'],
 ['09/15/15 01:00:00 PM', 10.32, 'Site1'],
 ['09/15/15 02:00:00 PM', 10.443, 'Site1'],
 ['09/15/15 03:00:00 PM', 10.492, 'Site1'],
 ['09/15/15 04:00:00 PM', 10.565, 'Site1'],
 ['09/15/15 05:00:00 PM', 10.614, 'Site1'],
 ['09/15/15 06:00:00 PM', 10.663, 'Site1'],
 ['09/15/15 07:00:00 PM', 10.687, 'Site1'],
 ['09/15/15 08:00:00 PM', 10.687, 'Site1']]

In [12]:
data[-10:]

[['10/20/15 01:00:00 AM', 11.686, 'Site6'],
 ['10/20/15 02:00:00 AM', 11.492, 'Site6'],
 ['10/20/15 03:00:00 AM', 11.346, 'Site6'],
 ['10/20/15 04:00:00 AM', 11.224, 'Site6'],
 ['10/20/15 05:00:00 AM', 11.175, 'Site6'],
 ['10/20/15 06:00:00 AM', 11.151, 'Site6'],
 ['10/20/15 07:00:00 AM', 11.127, 'Site6'],
 ['10/20/15 08:00:00 AM', 11.127, 'Site6'],
 ['10/20/15 09:00:00 AM', 11.078, 'Site6'],
 ['10/20/15 10:00:00 AM', 11.321, 'Site6']]

In [13]:
data[800:900]

[['10/18/15 07:00:00 PM', 10.736, 'Site1'],
 ['10/18/15 08:00:00 PM', 10.761, 'Site1'],
 ['10/18/15 09:00:00 PM', 10.761, 'Site1'],
 ['10/18/15 10:00:00 PM', 10.785, 'Site1'],
 ['10/18/15 11:00:00 PM', 10.785, 'Site1'],
 ['10/19/15 12:00:00 AM', 10.785, 'Site1'],
 ['10/19/15 01:00:00 AM', 10.785, 'Site1'],
 ['10/19/15 02:00:00 AM', 10.785, 'Site1'],
 ['10/19/15 03:00:00 AM', 10.81, 'Site1'],
 ['10/19/15 04:00:00 AM', 10.81, 'Site1'],
 ['10/19/15 05:00:00 AM', 10.81, 'Site1'],
 ['10/19/15 06:00:00 AM', 10.81, 'Site1'],
 ['10/19/15 07:00:00 AM', 10.834, 'Site1'],
 ['10/19/15 08:00:00 AM', 10.834, 'Site1'],
 ['10/19/15 09:00:00 AM', 10.834, 'Site1'],
 ['10/19/15 10:00:00 AM', 10.858, 'Site1'],
 ['10/19/15 11:00:00 AM', 10.883, 'Site1'],
 ['10/19/15 12:00:00 PM', 10.932, 'Site1'],
 ['10/19/15 01:00:00 PM', 10.956, 'Site1'],
 ['10/19/15 02:00:00 PM', 10.956, 'Site1'],
 ['10/19/15 03:00:00 PM', 10.98, 'Site1'],
 ['10/19/15 04:00:00 PM', 10.98, 'Site1'],
 ['10/19/15 05:00:00 PM', 11.005, 'Sit

Finally, how to write this list of lists to a CSV file?
Googled "python write list of lists to csv" and found this:
http://stackoverflow.com/questions/14037540/writing-a-python-list-of-lists-to-a-csv-file. You'll notice that the solution is very similar to what we did when reading from a CSV file using the `csv` library. Not surprising.


In [15]:
import csv
# Write CSV file
with open('data/sites_1_6_openpyxl.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerows(data)

Now that we've got all the pieces working, let's put it all together as a function. Our function will have two input arguments:

* a string representing the XLSX filename to process
* a string representing the CSV filename to create

Instead of doing this in a notebook, I did it in Spyder. I wanted to be able to use a debugger and some of the other nice features of an IDE. Let's start up Spyder and I'll talk us through getting the correct file open (*reading_excel_temperature_data.py*). I've put this approach together as a function. See https://openpyxl.readthedocs.io/en/default/index.html for more about the capabilities of **openpyxl**.